# The notebook is a local instance of RNA-seq workshop @ ASHG for Jupyter 
## to be run locally with GRC local mounts within network (secure)
A hands-on rare disease diagnostic workshop on RNA-seq data #ASHG20

The accompanying presentation can be found here: tinyurl.com/RNA-ASHG-presentation.

GRC jupyter server: krakatoa.grid.gs.washington.edu:47897

use tunnel: ssh -L 127.0.0.1:47897:krakatoa.grid.gs.washington.edu:47897 nexus.gs.washingon.edu

open browser to 127.0.0.1:47897

password: bay4


In [1]:
#Use R
IRkernel::installspec()

# check the environment that this notebook is using:

In [2]:
Sys.getenv()

_LMFILES_               /usr/share/Modules/modulefiles/modules:/usr/share/Modules/modulefiles/modules-init:/usr/share/Modules/modulefiles/modules-gs/prod:/net/gs/vol3/software/modules-repo/CentOS7/python/3.6.5:/net/gs/vol3/software/modules-repo/python/3.6.5/CentOS7/numpy/1.15.1:/net/gs/vol3/software/modules-repo/python/3.6.5/CentOS7/pandas/1.3.1:/net/gs/vol3/software/modules-repo/python/3.6.5/CentOS7/scipy/1.1.0:/net/gs/vol3/software/modules-repo/python/3.6.5/CentOS7/scikit-learn/0.23.1
ANALYSISDIR             /net/grc/vol6/mendelian_projects/mendelian_analysis/mcgold/
ARCHPATH                Linux/CentOS7/x86_64
BASH_FUNC_module()      () { eval `/usr/bin/modulecmd bash $*` }
CLINVAR_FILE            /net/grc/vol6/mendelian_projects/mendelian_analysis/uwcmg_reanalysis_support_files/annotation/ClinVar_current.vcf.gz
CLINVAR_GIP_FILE        /net/grc/vol6/mendelian_projects/mendelian_analysis/uwcmg_reanalysis_support_files/annotation/ClinVar_geneIntervalPhenotypes_latest.bed.gz
CLOUDSDK_P

In [39]:
Sys.getenv('LD_LIBRARY_PATH')

[1] "/net/gs/vol3/software/modules-sw/R/4.0.3/Linux/CentOS7/x86_64/lib64/R/lib:/usr/local/lib64:/usr/lib/jvm/java-14-openjdk-14.0.2.12-1.rolling.el7.x86_64/lib/server:/net/gs/vol3/software/modules-sw/python/3.6.5/Linux/CentOS7/x86_64/lib/:/nfs/home/mcgold/bin/lib64:"

In [4]:
# set you local library path for user-specific installs

In [5]:
# module library path:
.libPaths(c("~/R/x86_64-pc-linux-gnu-library/4.0",'/net/gs/vol3/software/modules-sw/R/4.0.3/Linux/CentOS7/x86_64/lib64/R/lib:/usr/local/lib64','/usr/lib/jvm/java-14-openjdk-14.0.2.12-1.rolling.el7.x86_64/lib/server','/net/gs/vol3/software/modules-sw/R/4.0.3/Linux/CentOS7/x86_64/lib64/R/lib','/net/gs/vol3/software/modules-sw/python/3.7.7/Linux/CentOS7/x86_64/lib/','/nfs/home/mcgold/bin/lib64'))

In [7]:
# make sure you make a place for your personal library folder 
# do: mkdir -p "~/R/x86_64-pc-linux-gnu-library/4.0" 

# Patch in the Local Installs instead of the cloud installs
## Only install this once

# annotables

## OUTRIDER
https://www.bioconductor.org/packages/release/bioc/html/OUTRIDER.html

# data.table

# ggplot2 as part of tidyverse

# ggpubr

# FRASER

# If you see an error or warning in installing packages please check that you added the local library directory with .libpaths, and have used a local install with the directory permissions for the local libraries set and present


.libPaths("~/R/x86_64-pc-linux-gnu-library/4.0")


# load the necessary installed libraries 

In [63]:
# outlier analysis
library(OUTRIDER)
library(annotables)
library(data.table)
library(ggplot2)
library(ggpubr)
# splice analysis
library(FRASER)
register(SerialParam())

In [68]:
setwd("/net/grc/vol6/mendelian_projects/udn_ucla_external_uwcmg_fam287604_rna_1/")

# Expression outlier detection:

Data import
First, we have to load the OUTRIDER package that we will use to run the full expression outlier analysis. To map ENSEMBL gene names and HGNC symbols we use annotables. For efficient data handling and visualization, we further load data.table, ggplot2, and ggpubr.



In [46]:
# sample annotation and raw read counts
#anno <- fread("./annotation.tsv")[, 1:6]
anno <- fread("/net/grc/vol6/mendelian_projects/udn_ucla_external_uwcmg_fam287604_rna_1/fam297604_annotation.tsv")
#cts  <- as.matrix(read.table("./outrider/raw_counts.tsv.gz"))
cts  <- as.matrix(read.table("/net/grc/vol6/mendelian_projects/udn_ucla_external_uwcmg_fam287604_rna_1/fam297604_counts.tsv"))
# look into the data
head(anno)
print("Dimensions of the annotation:")
dim(anno)
dim(cts)
cts
print("Dimensions of the count table:")
dim(cts)

Warning message in fread("/net/grc/vol6/mendelian_projects/udn_ucla_external_uwcmg_fam287604_rna_1/fam297604_annotation.tsv"):
“Found and resolved improper quoting in first 100 rows. If the fields are not quoted (e.g. field separator does not appear within any field), try quote="" to avoid this warning.”


"""RNA_ID""","""SEX""","""ORIGIN""","""LAB""","""INDIVIDUAL""","""phase1TG""","""OUTRIDER_GROUP""","""PAIRED_END""","""COUNT_MODE""","""COUNT_OVERLAP""","""COUNT_AS_STRANDED""","""sampleID"""
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<lgl>,<chr>,<lgl>,<chr>,<chr>
"""s307256""","""female""","""blood""","""UW""","""s307256""",1,"""blood""",TRUE,"""IntersectionStrict""",TRUE,"""yes""","""s307256"""
"""s307257""","""female""","""fibroblast""","""UW""","""s307257""",1,"""fibroblast""",TRUE,"""IntersectionStrict""",TRUE,"""yes""","""s307257"""
"""s307258""","""female","""blood""","""UW""","""s307258""",1,"""blood""",TRUE,"""IntersectionStrict""",TRUE,"""yes""","""s307258"""
"""s307259""","""female""","""fibroblast""","""UW""","""s307259""",1,"""fibroblast""",TRUE,"""IntersectionStrict""",TRUE,"""yes""","""s307259"""
"""s307260""","""male""","""fibroblast""","""UW""","""s307260""",1,"""fibroblast""",TRUE,"""IntersectionStrict""",TRUE,"""yes""","""s307260"""


[1] "Dimensions of the annotation:"


[1]  5 12

[1] 58219     5

,s307256,s307257,s307258,s307259,s307260
ENSG00000223972.5,5,6,0,0,1
ENSG00000227232.5,105,156,101,192,81
ENSG00000278267.1,0,1,0,0,0
ENSG00000243485.5,0,1,0,1,1
ENSG00000284332.1,0,0,0,0,0
ENSG00000237613.2,0,0,0,0,0
ENSG00000268020.3,0,0,0,0,0
ENSG00000240361.1,0,0,0,0,0
ENSG00000186092.4,0,0,0,0,0
ENSG00000238009.6,18,0,11,0,0


[1] "Dimensions of the count table:"


[1] 58219     5

In [50]:
#cts

In [54]:
#anno[,sampleID:=INDIVIDUAL]
ods3 <- OutriderDataSet(countData=cts, colData=anno)
ods3

Warning message in OutriderDataSet(countData = cts, colData = anno):
“No sampleID was specified. We will generate a generic one.”


class: OutriderDataSet
class: RangedSummarizedExperiment 
dim: 58219 5 
metadata(1): version
assays(1): counts
rownames(58219): ENSG00000223972.5 ENSG00000227232.5 ...
  ENSG00000210195.2 ENSG00000210196.2
rowData names(0):
colnames(5): s307256 s307257 s307258 s307259 s307260
colData names(13): "RNA_ID" "SEX" ... "sampleID" sampleID

# counts based analyses use a size factor to control for sequencing effort - estimate that
Size factor
One important metric is the sizeFactor. The sizeFactor represents the sequencing depth of each sample with respect to the others and is centered around 1. The sizeFactor can be estimated with estimateSizeFactors based on the DESeq2 function. In gene expression analysis, one of the first normalization steps is done using the sizeFactor.



In [69]:
# estimate sizeFactors 
ods2 <- estimateSizeFactors(ods3)

# visualize the sizeFactors
plotSizeFactors(ods2)

ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [65]:
#Let's have a look at the samples with the lowest sizeFactor.
round(sort(sizeFactors(ods2)), digits=2)[1:5]

s307256 s307258 s307260 s307257 s307259 
   0.78    0.88    1.01    1.22    1.25

# Filter non-expressed genes
Not all genes are expressed in a given tissue/sample. It depends on the tissues and the sequencing depth how many expressed genes are expected.

To make the modelling and outlier detection more robust, a good filtering of non-expressed genes is crucial. The cutoffs are dependent on the project and should be defined beforehand. The FPKM value (fragments per kilobase of million mapped reads) is a good measurement for this. [TPM is better]

Here, we keep genes where at least 5% of the samples have an FPKM/TPM value greater than 1.

To compute the FPKM value, we need the gene length from the annotation file.

In [61]:
# load gene annotation to compute FPKM values
txdb <- loadDb("annotations/gencode.v29lift37.annotation.txdb")
ods1 <- filterExpression(ods2, gtfFile=txdb, filterGenes=FALSE)

Warning message in computeGeneLength(object, gtfFile = gtfFile, ...):
“Some genes (n=52897) are not found in the annotation. Setting 'basepairs' == 1. The first 6 names are:
ENSG00000223972.5, ENSG00000227232.5, ENSG00000278267.1, ENSG00000243485.5, ENSG00000284332.1, ENSG00000237613.2”
25097 genes did not pass the filter. This is 43.1% of the genes.



In [66]:
# FPKM distribution across all sample/gene pairs
plotFPKM(ods1) + theme(legend.position = 'bottom')

173187 sample-gene combinations are zero. This is 59.5% of the data



ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [70]:
# filter object based on the expression status of the genes
ods <- ods1[mcols(ods1)[,"passedFilter"],]
ods

class: OutriderDataSet
class: RangedSummarizedExperiment 
dim: 33122 5 
metadata(1): version
assays(1): counts
rownames(33122): ENSG00000223972.5 ENSG00000227232.5 ...
  ENSG00000210195.2 ENSG00000210196.2
rowData names(3): loggeomeans passedFilter basepairs
colnames(5): s307256 s307257 s307258 s307259 s307260
colData names(19): sampleID X.RNA_ID. ... passedFilterGenes
  expressedGenesRank

In [72]:
#Sample co-variation (especially when the same samples or fraction of relevant response variables go to different places be careful!)
# Usually in RNA-seq data one can see sample co-variation and hence should control for that. 
# In most of the cases one knows the cause of the structure (eg. sex, origin, batch) but 
# sometimes it comes from unkown confounders. For more details see slides.

# Make heatmap figure bigger
options(repr.plot.width=6, repr.plot.height=5)

# use normalize=FALSE since the data is not yet corrected
# use columns from the annotation to add labels to the heatmap
plotCountCorHeatmap(ods, colGroups=c("ORIGIN"), 
        rowGroups="LAB", normalize=FALSE)

ERROR: Error: subscript contains invalid names


In [74]:
colnames(anno)

[1] "\"RNA_ID\""            "\"SEX\""               "\"ORIGIN\""           
 [4] "\"LAB\""               "\"INDIVIDUAL\""        "\"phase1TG\""         
 [7] "\"OUTRIDER_GROUP\""    "\"PAIRED_END\""        "\"COUNT_MODE\""       
[10] "\"COUNT_OVERLAP\""     "\"COUNT_AS_STRANDED\"" "\"sampleID\""

In [80]:
#Detection of expression outlier events
#We have loaded the data, did some quality control, and preprocessed (filtered) the data. Now we can start with the normalization and outlier detection.

#Using a denoising autoencoder to correct for sample co-variations.

#Fitting the model
#We will now use OUTRIDER to model the sample co-variation, also called latent space, based on the given gene expression data. Using the correct dimension for the latent space is crucial for achieving the best performance. Finding the right dimension can be done with findEncodingDim().

# use BiocParallel to compute in parallel
register(MulticoreParam(workers=2, tasks=10, progressbar=TRUE))
#​
## As this step is heavy to compute we will use a prefitted model.
## If you use your own data please uncomment it and run it.
## It can take 30 minutes to complete depending on the sample/gene size.
#ods <- readRDS("outrider/fitted_ods.RDS")
#​
## Find the optimal dimension for the latent space
# ods <- findEncodingDim(ods)
#getBestQ(ods)
#plotEncDimSearch(ods)
#​
## fit the OUTRIDER model
 ods <- OUTRIDER(ods, q=3)
#Let's see how well the denoising autoencoder fitted the latent space of our data. Ideally, we should not see any cluster anymore and most of the plot should be rather white than red or blue except for the diagonal. Compare the dendogram with the previous heatmap. What do you observe?

plotCountCorHeatmap(ods, colGroups=c("SEX", "ORIGIN"), 
        rowGroups="LAB", normalize=TRUE)

Thu Jan 28 03:56:37 2021: SizeFactor estimation ...

Thu Jan 28 03:56:37 2021: Controlling for confounders ...

Thu Jan 28 03:56:37 2021: Using the autoencoder implementation for controlling.



[1] "Thu Jan 28 03:56:47 2021: Initial PCA loss: 3.94498453926547"
  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

[1] "Thu Jan 28 04:44:00 2021: Iteration: 1 loss: 2.72834123481866"
  |======================================================================| 100%

  |======================================================================| 100%

[1] "Thu Jan 28 05:07:14 2021: Iteration: 2 loss: 2.72556709952805"
  |======================================================================| 100%

  |======================================================================| 100%

[1] "Thu Jan 28 05:30:38 2021: Iteration: 3 loss: 2.72466359815047"
  |======================================================================| 100%



Thu Jan 28 10:11:24 2021: Used the autoencoder implementation for controlling.

Thu Jan 28 10:11:24 2021: P-value calculation ...



  |======================================================================| 100%



Thu Jan 28 10:23:12 2021: Zscore calculation ...



ERROR: Error: subscript contains invalid names


Detection of expression outliers
After fitting the model and correcting for sample co-variations, we can now obtain our expression outliers. Here, we define an outlier as an event that significantly deviates from the expected Negative-Binomial distribution after controlling for confounders. For details see slides.

no image
The two-sided p value is defined as:

𝑃(𝑘𝑖𝑗)=2⋅min(12,∑𝑘𝑖𝑗𝑘=0𝑁𝐵(𝑘𝑖𝑗|𝜇𝑖𝑗),𝜃𝑗),1−∑𝑘𝑖𝑗𝑘=0𝑁𝐵(𝑘𝑖𝑗|𝜇𝑖𝑗,𝜃𝑗)) ,

with a genome-wide significance threshold of  𝐹𝐷𝑅<0.05 .

Let's check how many expression outliers per sample were found.



In [76]:
# an event that significantly deviates from the expected Negative-Binomial distribution after controlling for confounders
plotAberrantPerSample(ods)

ERROR: Error in checkFullAnalysis(object): Please calculate the P-values before calling the 'IRkernel::main' function. Please do: ods <- computePvalues(ods)


Z score approach
There are many different ways to define outliers. One common approach is to compute a Z score and then define outliers as  |𝑧|>3  or  |𝑧|>2 . In our OUTRIDER publication, we recommend to use a statistical assesment for defining outliers over the Z score approach.

Despite our recommendation, one can use OUTRIDER to retrieve Z score derived outliers by setting padjCutoff=1 and zScoreCutoff=2 or another value of choice.



In [77]:
plotAberrantPerSample(ods, padjCutoff=1, zScoreCutoff=2, yadjust=c(1.3,1.4))

ERROR: Error in checkFullAnalysis(object): Please calculate the P-values before calling the 'IRkernel::main' function. Please do: ods <- computePvalues(ods)


In [78]:
#in order to make the results table more intuitive, we now change the ENSEMBL gene IDs into HGNC symbols. For this we use the annotables package.

# remove versioning of gene IDs and merge with GRCh37 from annotables
geneIDs <- gsub("\\.[0-9]*(_[0-9]*)?.*$", "", rownames(ods))
map <- merge(data.table(ensgene=geneIDs), grch37, sort=FALSE,
        all.x=TRUE)[!duplicated(ensgene),]
#​
# set new gene names only if hgnc symbol is present
if(!"ENSG" %in% colnames(mcols(ods))){
    mcols(ods)$ENSG <- geneIDs
    rownames(ods) <- map[,ifelse(
            is.na(symbol) | symbol == "" | duplicated(symbol), geneIDs, symbol)]
}
#Now we can retrieve our expression outliers using the results function.



In [ ]:
res <- results(ods)
head(res)
dim(res)

In [ ]:
# volcano plot of sample NA18873
plotVolcano(ods, "NA18873", base=TRUE)

In [ ]:
# subset results for this patient
res[sampleID == "NA18873"]

In [ ]:
# make the plot again default size
options(repr.plot.width=8, repr.plot.height=4)

ggarrange(ncol=2,
    plotExpressionRank(ods, "TIMMDC1", norm=FALSE, basePlot=TRUE) + scale_y_log10(lim=c(300,2000)),
    plotExpressionRank(ods, "TIMMDC1", norm=TRUE,  basePlot=TRUE) + scale_y_log10(lim=c(300,2000)))

In [ ]:
# set again the default size for the plot
options(repr.plot.width=4, repr.plot.height=4)

plotQQ(ods, "TIMMDC1")

In [ ]:
# make the plot wider
options(repr.plot.width=8, repr.plot.height=4)

# Volcano plot with significance based on p value or on Z score threshold
ggarrange(ncol=2,
    plotVolcano(ods, "NA11918", main="FDR (<0.05) based volcano plot", 
            base=TRUE, padjCutoff=0.05, zScoreCutoff=0),
    plotVolcano(ods, "NA11918", main="Z score (>3) based volcano plot",
            base=TRUE, padjCutoff=1, zScoreCutoff=3))

In [ ]:
# subset results for this patient (p value based)
res[sampleID == "NA11918"]

In [ ]:
resZscore <- results(ods, padjCutoff=1, zScoreCutoff=3)
resZscore[sampleID == "NA11918"]

# ------------------- Outlier Analysis Done! -------------------------------------

# Aberrant Splicing Detection in RNA-seq data
A short introduction to splicing can be found in presentation here.

# Data import
First, we have to load the FRASER package that we will use to run the full aberrant splicing analysis. For efficient data handling and visualization we further load data.table, ggplot2, and ggpubr.

In [ ]:
library(FRASER)
library(data.table)
library(ggplot2)
library(ggpubr)
register(SerialParam())

In [ ]:
# load gene annotation to compute FPKM values
txdb <- loadDb("annotations/gencode.v29lift37.annotation.txdb")
ods <- filterExpression(ods, gtfFile=txdb, filterGenes=FALSE)

In [ ]:
# load and prepare annotation (add needed columns for FRASER)
anno <- fread("annotation.tsv")[,1:6]
anno[, sampleID:=INDIVIDUAL]
## only needed if you start from BAM files and need to count reads first
# anno[, bamFile:="<path/to/sample/bam/file>"]

# get raw counts (remove first column since it is the row index)
junctionCts   <- fread("./splicing/raw_junction_counts.tsv.gz")
spliceSiteCts <- fread("./splicing/raw_site_counts.tsv.gz")

In [ ]:
anno[,sampleID:=INDIVIDUAL]
ods <- OutriderDataSet(countData=cts, colData=anno)
ods

In [ ]:
head(anno)
junctionCts[1:6,1:15]
spliceSiteCts[1:6,1:15]

In [ ]:
# create FRASER object
fds <- FraserDataSet(colData=anno, junctions=junctionCts, 
        spliceSites=spliceSiteCts)
fds

In [ ]:
# randomly subset samples 
set.seed(42)
patient_ids <- c("NA11918", "NA20505", "HG00132")
fds <- fds[
    seqnames(fds) %in% c("chr3", "chr6", "chr19"),
    unique(c(patient_ids, sample(colnames(fds))))[1:60]]
dontWriteHDF5(fds) <- TRUE
fds

## Data preprocessing and QC

As with gene expression analysis, a good quality control of the raw data is crucial. As we will not go into detail here, please refer for some hints to the [gene expression slides](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit?pli=1#slide=id.g64b01c344e_4_51).

### Filtering the data

Before we can filter the data, we have to compute the main splicing metric: the $\psi$-values (Percent Spliced In) and $\theta$-values (splicing efficiency).

In [ ]:
fds <- calculatePSIValues(fds) # take 1-2 minutes

In [ ]:
fds <- filterExpressionAndVariability(fds, minDeltaPsi=0.1, filter=FALSE)

In [ ]:
# make the plot wider
options(repr.plot.width=5, repr.plot.height=4)

plotFilterExpression(fds, bins=100)
plotFilterVariability(fds) + theme(legend.position="none")

After looking at the expression distribution between filtered and unfiltered junctions, we can now subset the dataset. We mainly see that we discarded junctions with overall low expression, which was the intention.

In [ ]:
fds <- fds[mcols(fds, type="j")[,"passed"],]
fds

### Sample co-variation

Since $\psi$ values are ratios within a sample, one might think that there should not be as much correlation structure as observed in gene expression data within the splicing data.

This is not true as we do see strong sample co-variation within many tissues and cohorts. Let's have a look into our data to see if we do have correlation structure or not. To have a better estimate, we use the logit transformed $\psi$ values to compute the correlation. We can annotate our plot with any information provided through the sample annotation.

In [ ]:
# I think this gets at detectability given degredation of 5' ends of samples DJM

In [ ]:
# Make heatmap figure bigger
options(repr.plot.width=6, repr.plot.height=5)

plotCountCorHeatmap(fds, type="psi5", logit=TRUE, sampleClustering=NA, 
        annotation_col=c("SEX", "LAB", "ORIGIN"))
plotCountCorHeatmap(fds, type="psi3", logit=TRUE, sampleClustering=NA, 
        annotation_col=c("SEX", "LAB", "ORIGIN"))

## Detection of aberrant splicing events

After preprocessing the raw data and visualizing it, we can start our analysis. Let's start with the first step in aberrant splicing detection: the model fitting.


### Fitting the splicing model

During the fitting procedure, we will normalize the splicing data and correct for confounding effects by using a denoising autoencoder. Using the correct dimension for the latent space is crucial to have the best performance. Finding the right dimension can be done with `optimHyperParams()`.

In [ ]:
# use BiocParallel to compute in parallel
register(MulticoreParam(workers=2, tasks=10, progressbar=TRUE))

## As this step is heavy to compute, we will use a prefitted model.
## If you use your own data please uncomment it and run it.
## It can take 30 minutes to complete depending on the sample/gene size.
## fds <- readRDS("splicing/fitted_fds.RDS")

## Find the optimal dimension for the latent space # I ran this see below
#for(i in c('psi3','psi5')){
#    fds <- optimHyperParams(fds, i)
#    bestQ(fds, i)
#    plotEncDimSearch(fds, i)
#}

## fit the FRASER model
fds <- FRASER(fds)

Let's see how well the correction worked by looking at the correlation heatmap again.

In [ ]:
plotCountCorHeatmap(fds, type="psi5", normalized=TRUE, logit=TRUE, topN=15000, 
        annotation_col=c("SEX", "LAB", "ORIGIN"), sampleClustering=NA)

### Calling splicing outliers

For aberrant splicing detection, we assume a beta binomial distribution and call outliers based on the significance level. But again the user can choose between a p value cutoff, a Z score cutoff or both. We do recommend to use a pure p value cutoff.

Let's have a look how many splicing outliers we detect by sample.

In [ ]:
plotAberrantPerSample(fds)

Again, before we extract the results let's add the interpretable HGNC symbols. *FRASER* comes already with an annotation function. The function uses a user provided TxDb object (defaults to *UCSC.hg19*) to overlap the genomic ranges with the known HGNC symbols. 

In [ ]:
# annotate
#fds <- annotateRangesWithTxDb(fds,txdb = txdb, orgDb = 'org.Hs.eg.db')
#fds <- annotateRangesWithTxDb(fds,featureName="hgnc_symbol_37", GRCh=37)
#require(TxDb.Hsapiens.UCSC.hg19.knownGene)
#txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
#require(org.Hs.eg.db)
#orgDb <- org.Hs.eg.db
#fds <- annotateRangesWithTxDb(fds, txdb=txdb, orgDb=orgDb)

# annotate
fds <- annotateRangesWithTxDb(fds)

# retrive results
register(SerialParam())
res <- as.data.table(results(fds))
res

As FRASER tests $\psi_5$, $\psi_3$ and $\theta$, it can reports multiple hits for the same event. This is helpful later when interprating the results. But of a first overview this can be overwhelming. Hence, we suggest to aggregrate the results first at the gene level before going into details.

In [ ]:
# convert back to GRange object
resAsGR <- makeGRangesFromDataFrame(res, keep.extra.columns = TRUE)

# group results by genes/sample
results_by_genes <- as.data.table(resultsByGenes(resAsGR))
results_by_genes


Now we can have some cut-offs to filter down the number of junctions we want to test later on.

Currently, we keep only junctions that support the following:
* At least one sample has 20 reads
* 5% of the samples have at least 1 read

Further, one could filter for:
* At least one sample has to have $|\Delta\psi| > 0.1$